In [14]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from datetime import datetime
import psycopg2

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException


# Configura o serviço a driver do Chrome
servico = Service(ChromeDriverManager().install())
# Inicia o navegador Chrome
navegador = webdriver.Chrome(service=servico)

navegador.get("http://127.0.0.1:5500/index.html")


In [15]:
# Carrega os dados do Excel usando Pandas
dados_excel = pd.read_excel(r"C:\Users\bruna\Documents\2024\PROJETOS\AUTOMAÇÕES\EXCEL PARA WEB\Base\Loja.xlsx")

In [16]:
# Conexão com o banco de dados
conn = psycopg2.connect(
    dbname="AutomacaoWeb",
    user="postgres",
    password="1234",
    host="localhost",
    port=""
)

In [17]:
for indice, linha in dados_excel.iterrows():
    # Abre o formulário da web
    navegador.get("http://127.0.0.1:5500/index.html")
    
    campo_codigo = navegador.find_element('xpath','//*[@id="codigo"]')
    valor_codigo = str(linha["COD Venda"])
    campo_codigo.send_keys(valor_codigo)
    print("Campo Código:", valor_codigo)
    
    campo_nome = navegador.find_element('xpath','//*[@id="nome"]')
    valor_nome = str(linha["Nome do Jogo"])
    campo_nome.send_keys(valor_nome) 
    print("Campo Nome:", valor_nome)

    campo_console = navegador.find_element('xpath','//*[@id="console"]')
    valor_console = str(linha["Console"])
    campo_console.send_keys(valor_console)
    print("Campo Console:", valor_console)

    campo_genero = navegador.find_element('xpath','//*[@id="genero"]')
    valor_genero = str(linha["Gênero"])
    campo_genero.send_keys(valor_genero)
    print("Campo Gênero:", valor_genero)

    campo_ano_lancamento = navegador.find_element('xpath','//*[@id="ano_lancamento"]')
    valor_ano_lancamento = str(linha["Ano de Lançamento"])
    campo_ano_lancamento.send_keys(valor_ano_lancamento)
    print("Campo Ano de Lançamento:", valor_ano_lancamento)

    campo_dlc = navegador.find_element('xpath','//*[@id="dlc"]')
    valor_dlc = str(linha["Jogo Possui DLC?"])
    campo_dlc.send_keys(valor_dlc)
    print("Campo Ano de Lançamento:", valor_dlc)

    campo_online = navegador.find_element('xpath','//*[@id="online"]')
    valor_online = str(linha["Disponível para Jogar Online?"])
    campo_online.send_keys(valor_online)
    print("Campo Disponível para Jogar Online?:", valor_online)

    campo_data_venda = navegador.find_element('xpath','//*[@id="data_venda"]')
    valor_data_venda = str(linha["Data de Venda"])
    campo_data_venda.send_keys(valor_data_venda)
    print("Campo Data de Venda:", valor_data_venda)

    campo_preco = navegador.find_element('xpath','//*[@id="preco"]')
    valor_preco = linha["Preço"].replace('$', '')
    campo_preco.send_keys(str(valor_preco))
    print("Campo Preço:", valor_preco)

    valor_quantidade = linha["Quantidade"]
    
    # Esperar até que o campo de quantidade esteja visível novamente
    try:
        campo_quantidade = WebDriverWait(navegador, 10).until(
            EC.visibility_of_element_located((By.XPATH, '//*[@id="quantidade"]'))
        )

        # Interagir com o campo de quantidade
        campo_quantidade.clear()  # Limpar o campo antes de enviar o valor
        campo_quantidade.send_keys(valor_quantidade)
        print("Campo Quantidade:", valor_quantidade)

    except StaleElementReferenceException:
        # Handle StaleElementReferenceException here, e.g., by refreshing the page
        navegador.refresh()
        # Or you can wait for the element to become clickable again
        campo_quantidade = WebDriverWait(navegador, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="quantidade"]'))
        )
        campo_quantidade.clear()
        campo_quantidade.send_keys(valor_quantidade)
        print("Campo Quantidade:", valor_quantidade)

    # Insere os dados no banco de dados PostgreSQL
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO vendas_jogos (codigo, nome, console, genero, ano_lancamento, possui_dlc, online, data_venda, preco, quantidade) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)",
        (valor_codigo, valor_nome, valor_console, valor_genero, valor_ano_lancamento, str(valor_dlc), str(valor_online), valor_data_venda, valor_preco, valor_quantidade)
    )

    conn.commit()
    cursor.close()

    # Envie o formulário
    botao_enviar = navegador.find_element('xpath','//button[@type="submit"]')
    botao_enviar.click()
    
    # Aguarde um pouco para que o formulário seja processado (você pode ajustar o tempo conforme necessário)
    navegador.implicitly_wait(15)


Campo Código: 1
Campo Nome: A Plague Tale: Innocence
Campo Console: Xbox One
Campo Gênero: Ação e Aventura
Campo Ano de Lançamento: 2019
Campo Ano de Lançamento: Sim
Campo Disponível para Jogar Online?: Não
Campo Data de Venda: 01.01.2019
Campo Preço: 34.99
Campo Quantidade: 432
Campo Código: 2
Campo Nome: Baldur’s Gate II: Shadows of Amn
Campo Console: PC
Campo Gênero: RPG
Campo Ano de Lançamento: 2000
Campo Ano de Lançamento: Não
Campo Disponível para Jogar Online?: Não
Campo Data de Venda: 01.02.2019
Campo Preço: 9.99
Campo Quantidade: 779
Campo Código: 3
Campo Nome: Batman: Arkham City
Campo Console: Xbox 360
Campo Gênero: Ação e Aventura
Campo Ano de Lançamento: 2011
Campo Ano de Lançamento: Sim
Campo Disponível para Jogar Online?: Não
Campo Data de Venda: 01.03.2019
Campo Preço: 29.99
Campo Quantidade: 215
Campo Código: 4
Campo Nome: Blur
Campo Console: Xbox 360
Campo Gênero: Arcade Race
Campo Ano de Lançamento: 2010
Campo Ano de Lançamento: Sim
Campo Disponível para Jogar Online

In [18]:
# Após concluir o loop, você pode fechar o navegador
navegador.quit()
# Fecha a conexão com o banco de dados PostgreSQL
conn.close()